# Overview

This example shows how to extract a time series of images from Earth Engine, annotate them, and save them as a video.

# Initialize Earth Engine

We start by importing the [Earth Engine Python API](https://pypi.org/project/earthengine-api/) module.

In [ ]:
import ee

The following command initializes the Earth Engine Python API.

In [ ]:
ee.Initialize()

If the cell produces output that displays an error about needing to authenticate, open up the notebook entitled `01 - Setup auth credentials` and follow the instructions.

# Define Helper Functions

In [ ]:
def GetTileLayerUrl(ee_image_object):
    map_id = ee.Image(ee_image_object).getMapId()
    tile_url_template = "https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}"
    return tile_url_template.format(**map_id)

# Building an User Interface

In [ ]:
import datetime
import ipywidgets as widgets
import ipyleaflet  # an interactive mapping "widget"

## Define the Dataset to work with.

In [ ]:
datasets = [
    {
        'name':'Landsat 8 Collection 1 TOA',
        'base_collection':(
            ee.ImageCollection('LANDSAT/LC08/C01/T1_RT_TOA')
              .select(['B4','B3','B2'])
        ),
        'vis_params':{
            'min':0,
            'max':0.3,
        },
    },
    {
        'name':'Sentinel-1 GRD HV (radar)',
        'base_collection':(
            ee.ImageCollection('COPERNICUS/S1_GRD')
              .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HV'))
              .select(['HV'])
        ),
        'vis_params':{
            'min':-30,
            'max':-10,
        },
    },
    {
        'name':'Sentinel-1 GRD HH (radar)',
        'base_collection':(
            ee.ImageCollection('COPERNICUS/S1_GRD')
              .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HH'))
              .select(['HH'])
        ),
        'vis_params':{
            'min':-25,
            'max':0,
        },
    },
    {
        'name':'Sentinel-1 GRD VV (radar)',
        'base_collection':(
            ee.ImageCollection('COPERNICUS/S1_GRD')
              .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
              .select(['VV'])
        ),
        'vis_params':{
            'min':-25,
            'max':0,
        },
    },
    {
        'name':'Sentinel-1 GRD VH (radar)',
        'base_collection':(
            ee.ImageCollection('COPERNICUS/S1_GRD')
              .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
              .select(['VH'])
        ),
        'vis_params':{
            'min':-25,
            'max':0,
        },
    }
]

In [ ]:
def get_dataset_dropdown_dict():
    return dict([(datasets[i]['name'],i) for i in range(len(datasets))])

def get_base_collection():
    return datasets[dataset_dropdown.index]['base_collection']

def get_vis_params():
    return datasets[dataset_dropdown.index]['vis_params']

## Define User Interface Elements

In [ ]:
roi_dimension = widgets.IntSlider(
    value=1e4,
    min=1e2,
    max=2e4,
    description='ROI Size (m):',
    continuous_update=False,
)
roi_dimension

In [ ]:
# Define the map.
map1 = ipyleaflet.Map(
    center=(81.55303991947001, -45.57952880859376),
    zoom=8,
    layout={
        'height':'300px',
        'min_height':'200px',
        'min_width':'200px'
    },
)
map1.add_control(ipyleaflet.LayersControl())

# Define and add a Marker pin to the map.
center_marker = ipyleaflet.Marker(
    name='ROI Selection Marker',
    location=map1.center
)
map1 += center_marker

mosaic_layer_group = ipyleaflet.LayerGroup(layers=(), name='Mosaic Layer')
map1 += mosaic_layer_group

roi_layer_group = ipyleaflet.LayerGroup(layers=(), name='ROI Layer')
map1 += roi_layer_group
map1

In [ ]:
dataset_dropdown = widgets.Dropdown(
    options=get_dataset_dropdown_dict(),
    description='Dataset:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='300px')
)
dataset_dropdown

In [ ]:
start_datepicker = widgets.DatePicker(
    description='Start Date:',
    disabled=False,
    value=datetime.datetime(2017, 8, 8),
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='240px')
)
start_datepicker

In [ ]:
interval_length_dropdown = widgets.IntSlider(
    value=5,
    min=1,
    max=32,
    description='Interval Length [days]:',
    style={'description_width': 'initial'},
    #layout=widgets.Layout(width='200px'),
    continuous_update=False,
)
interval_length_dropdown

In [ ]:
num_intervals_slider = widgets.IntSlider(
    value=5,
    min=1,
    max=25,
    description='# intervals:',
    style={'description_width': 'initial'},
    continuous_update=False,
)
num_intervals_slider

In [ ]:
# Define an output widget (mainly for debugging messages)
out = widgets.Output()
out.clear_output()
out

## Display the UI Elements

In [ ]:
# Layout the UI elements.
panel = widgets.Box(
    [
        map1,
        widgets.VBox([
            dataset_dropdown,
            start_datepicker,
            interval_length_dropdown,
            num_intervals_slider,
            roi_dimension,
            #out
        ])
    ],
    layout=widgets.Layout(
        display='flex',
        flex_flow='row',
        flex_wrap='wrap'
    )
)
display(panel)

## Define interactions

In [ ]:
# Define helper functions to swap the coordinate ordering.
def swap_coordinate_xy_for_location(coord):
    return (coord[1],coord[0])

def swap_coordinate_xy_for_list(coord_list):
    return [swap_coordinate_xy_for_location(coord) for coord in coord_list]  

def update_roi_layer(map_reference):    
    coord_list_xy = get_roi_polygon()['coordinates'][0]
    coord_list_yx = swap_coordinate_xy_for_list(coord_list_xy)
    
    roi_layer = ipyleaflet.Polygon(
        name='TEST update ROI Polygon',
        locations=coord_list_yx,
        weight=3,
        color='#F00',
        opacity=0.8,
        fill_opacity=0.1,
        fill_color='#F00'
    )
    roi_layer_group.clear_layers()
    roi_layer_group.add_layer(roi_layer)

In [ ]:
def get_roi_polygon():
    center_marker_xy = swap_coordinate_xy_for_location(center_marker.location)
    centroid = ee.Geometry.Point(center_marker_xy)
    buffered = centroid.buffer(roi_dimension.value).bounds()
    return buffered.getInfo() 

In [ ]:
def get_composite_image():
    with out:
        print('DEBUG: starting get_composite_image()')
        
    start_date = start_datepicker.value
    interval_length = interval_length_dropdown.value
    num_intervals = num_intervals_slider.value
        
    base_collection = get_base_collection()
    end_date = start_date + datetime.timedelta(days=num_intervals*interval_length)
    
    with out:
        print('DEBUG: start_date =', start_date)
        print('DEBUG: num_intervals =', num_intervals)
        print('DEBUG: interval_length =', interval_length)
    
    # Create Earth Engine objects.
    ee_start_date = ee.Date(start_date.isoformat())
    ee_end_date = ee.Date(end_date.isoformat())
    
    return base_collection.filterDate(ee_start_date, ee_end_date).median()

In [ ]:
def get_image_collection():

#     with out:
#         print('DEBUG: starting get_image_collection()')
    
    # Get filter values from the UI widgets.
    roi = get_roi_polygon()
    start_date = start_datepicker.value
    interval_length = interval_length_dropdown.value
    num_intervals = num_intervals_slider.value
    
    base_collection = get_base_collection()
    
    ee_image_list = []
    for i in range(num_intervals):
        interval_start_date = start_date + datetime.timedelta(days=i*interval_length)
        interval_end_date = interval_start_date + datetime.timedelta(days=interval_length)   
        # Create Earth Engine objects.
        ee_start_date = ee.Date(interval_start_date.isoformat())
        ee_end_date = ee.Date(interval_end_date.isoformat())
        
        interval_collection = (
            base_collection
                .filterDate(ee_start_date, ee_end_date)
                .filterBounds(roi)
        )
        
        interval_image = interval_collection.mosaic()
        # Set metadata on the image.
        interval_image = (
            interval_image.set({'system:time_start':ee_start_date.millis()})
                          .set({'system:time_end':ee_end_date.millis()})
        )
        ee_image_list.append(interval_image)
            
    return ee.ImageCollection.fromImages(ee_image_list)

In [ ]:
def get_interval_list():
    start_date = start_datepicker.value
    interval_length = interval_length_dropdown.value
    num_intervals = num_intervals_slider.value
    
#     with out:
#         print('DEBUG: starting get_interval_list()')
        
    interval_list = []
    for i in range(num_intervals):
        interval_start_date = start_date + datetime.timedelta(days=i*interval_length)
        interval_end_date = interval_start_date + datetime.timedelta(days=interval_length)
        interval_list.append({'start_date':interval_start_date, 'end_date':interval_end_date})
    return interval_list

In [ ]:
def update_mosaic_layer(map_reference):

#     with out:
#         print('DEBUG starting update_mosaic_layer()')
#         print('Total images = {0}'.format(get_image_collection().size().getInfo()))
        
    mosaic_tilelayer = ipyleaflet.TileLayer(
        url=GetTileLayerUrl(
            get_composite_image().visualize(**get_vis_params())
        ),
        attribution='Map tiles by <a href="http://earthengine.google.com/">Earth Engine</a>.'
    )
    mosaic_layer_group.clear_layers()
    mosaic_layer_group.add_layer(mosaic_tilelayer)

In [ ]:
# Define the actions performed when the marker moves.
def center_marker_on_move(change):
#     with out:
#         print('DEBUG starting center_marker_on_move()')
    update_roi_layer(map1)
center_marker.unobserve_all()
center_marker.observe(center_marker_on_move, names='location')

In [ ]:
# Define the actions performed when the dataset dropdown has changed.
def dataset_dropdown_on_change(change):
#     with out:
#         print('DEBUG starting dataset_dropdown_on_change()')
    update_mosaic_layer(map1)
dataset_dropdown.unobserve_all()
dataset_dropdown.observe(dataset_dropdown_on_change, names='index')

In [ ]:
# Define the actions performed when the starting date is changed.
def start_datepicker_on_change(change):
#     with out:
#         print('DEBUG starting start_datepicker_on_change()')
#         print('start_datepicker = {0}'.format(start_datepicker.value))
    update_roi_layer(map1)
    update_mosaic_layer(map1)
start_datepicker.unobserve_all()
start_datepicker.observe(start_datepicker_on_change, names='value')

In [ ]:
# Define the actions performed when the ROI size is changed.
def roi_dimension_on_change(change):
#     with out:
#         print('DEBUG starting roi_dimension_on_change()')
    update_roi_layer(map1)

roi_dimension.unobserve_all()
roi_dimension.observe(roi_dimension_on_change, names='value')

In [ ]:
# Define the actions performed when the number of intervals has changed.
def num_intervals_on_change(change):
    with out:
        print('DEBUG starting num_intervals_on_change()')
        #print('num_intervals_slider.value = {0}'.format(num_intervals_slider.value))
#     update_roi_layer(map1)
    update_mosaic_layer(map1)

num_intervals_slider.unobserve_all()
num_intervals_slider.observe(num_intervals_on_change, names='value')

In [ ]:
# Define the actions performed when the interval length has changed.
def interval_length_on_change(change):
    with out:
        print('DEBUG starting num_intervals_on_change()')
        #print('interval_length = {0}'.format(interval_length_dropdown.value))
#     update_roi_layer(map1)
    update_mosaic_layer(map1)

interval_length_dropdown.unobserve_all()
interval_length_dropdown.observe(interval_length_on_change, names='value')

Initialize the overlay layers.

In [ ]:
update_roi_layer(map1)
update_mosaic_layer(map1)

# Building a (time) series

In [ ]:
import os
output_directory = 'output'
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

In [ ]:
interval_list = get_interval_list()

In [ ]:
sample_interval = interval_list[1]
sample_interval

In [ ]:
def create_mosaic_image(time_interval, projection):
    start_datetime = time_interval['start_date']
    end_datetime = time_interval['end_date']
    
    ee_start_date = ee.Date(start_datetime.isoformat())
    ee_end_date = ee.Date(end_datetime.isoformat())
        
    base_collection = get_base_collection()
    roi = get_roi_polygon()
    interval_collection = (
        base_collection
            .filterDate(ee_start_date, ee_end_date)
            .filterBounds(roi)
    )
    interval_image = interval_collection.mosaic().clip(roi)
    
#     # Set the projection to match the projection of the first image in the base collection.
#     projection = ee.Image(base_collection.filterBounds(roi).first()).projection()
    interval_image = interval_image.reproject(projection)

    # Return an empty image if the interval collection is empty.
    interval_image = ee.Image(ee.Algorithms.If(
        interval_collection.size().gt(0),
        interval_image,
        ee.Image().rename('HV')
    ))
    
    interval_image = (
        interval_image.set({'system:time_start':ee_start_date.millis()})
                      .set({'system:time_end':ee_end_date.millis()})
    )
    
    return interval_image

In [ ]:
# Test it out on a single image.
test_image = create_mosaic_image(sample_interval, 'EPSG:4326')
roi = get_roi_polygon()
from IPython.display import Image
test_url = test_image.visualize(**get_vis_params()).getThumbUrl({
    'region':get_roi_polygon()
})
Image(url=test_url, embed=True, format='png')

In [ ]:
import PIL.Image
import PIL.ImageDraw
import PIL.ImageFont
import requests

def annotate_image(image_url, date_string, outfile):
   
    # Create a base image, from the Earth Engine Thumbnail URL.
    base_ee = PIL.Image.open(requests.get(image_url, stream=True).raw).convert('RGBA')

    # Create a background image.
    background = PIL.Image.new('RGBA', base_ee.size, (128,128,128,255))
    
    # make a blank image for the text, initialized to transparent text color
    txt = PIL.Image.new('RGBA', base_ee.size, (255,255,255,0))
    # Get a font.
    # Additional fonts available on this server can be found by running the following in a terminal:
    # find / -name *.ttf
    text_size = 30
    fnt = PIL.ImageFont.truetype('/usr/share/fonts/truetype/liberation/LiberationMono-Bold.ttf', text_size)
    # Get a drawing context.
    d = PIL.ImageDraw.Draw(txt)
    # Draw text. For date formatting codes see:
    # https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior
    #date_string = '{:%Y-%m-%d}'.format(start_time)
    margin = 10
    d.text((margin, margin), date_string, font=fnt, fill=(255,0,0,255))
    d.text((margin, margin+text_size), 'ESIP rocks!', font=fnt, fill=(255,0,0,255))
    
    out_image = background
    # Add the EE data.
    out_image = PIL.Image.alpha_composite(out_image, base_ee)
    # Add the text annotation.
    out_image = PIL.Image.alpha_composite(out_image, txt)

    out_image.save(outfile, format='PNG')

    return out_image

In [ ]:
date_string = ee.Date(test_image.get('system:time_start')).format('YYYY-MM-dd').getInfo()
out_image = annotate_image(test_url, date_string, 'test.png')
display(out_image)

# Testing out a series

This section will demonstrate outputing an image time series for the specified location, time interval, and image collection.

In [ ]:
import glob

# Remove previously saved images.
for filename in glob.glob('output/*.png'):
    os.remove(filename)

In [ ]:
roi = get_roi_polygon()

# Set the projection to match the projection of the first image.
first_image_projection = (
    ee.Image(
        get_base_collection()
            .filterDate(ee.Date(interval_list[0]['start_date'].isoformat()), ee.Date('2100'))
            .filterBounds(roi)
            .first()
    ).projection()
)

image_no = 0
for interval in interval_list:
    image_no += 1
    test_image = create_mosaic_image(interval, first_image_projection)
    test_url = test_image.visualize(**get_vis_params()).getThumbUrl({
        'region':get_roi_polygon()
    })
    date_string = ee.Date(test_image.get('system:time_start')).format('YYYY-MM-dd').getInfo()
    outfile_name = '{0}/img{1:03d}.png'.format(output_directory, image_no)
    out_image = annotate_image(test_url, date_string, outfile_name)
    display(out_image)

## Create an animated GIF.

In [ ]:
import imageio

gif_images = []
filenames = sorted(glob.glob('output/img???.png'))
for filename in filenames:
    gif_images.append(imageio.imread(filename))
imageio.mimsave('output/movie.gif', gif_images, 'GIF', duration=0.5)

## Create an MPEG video.

In [ ]:
%%bash  
ffmpeg -r 1 \
    -i output/img%03d.png \
    -vcodec mpeg4 \
    -y output/out.mp4

In [ ]:
print('Done!')